<a href="https://colab.research.google.com/github/garjita63/stock-market-analytics-zoomcamp/blob/main/homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution of Question 1

In [1]:
import pandas as pd
import requests
from datetime import datetime

# Define function to extract price from "Price Range" string
def extract_price(price_range):
  if pd.isna(price_range):
    return None
  elif "-" not in price_range:
    # Remove leading/trailing spaces and dollar sign ($) before conversion (handle empty string)
    price_without_symbol = price_range.strip(" $")
    return float(price_without_symbol) if price_without_symbol else None
  else:
    prices = price_range.split("-")
    # Check if both prices are valid before converting and calculating average
    if all(price.strip(" $") for price in prices):
      return (float(prices[0].strip(" $")) + float(prices[1].strip(" $"))) / 2
    else:
      return None


url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url)

filings_dfs = pd.read_html(response.text)

filings_df = filings_dfs[0]

# Convert 'Filing Date' to datetime
filings_df['Filing Date'] = pd.to_datetime(filings_df['Filing Date'])

# Convert 'Shares Offered' to float
filings_df['Shares Offered'] = pd.to_numeric(filings_df['Shares Offered'].str.replace(',', ''), errors='coerce')

# Define 'Avg_price' column
filings_df['Avg_price'] = filings_df['Price Range'].apply(extract_price)

# Calculate 'Shares_offered_value'
filings_df['Shares_offered_value'] = filings_df['Shares Offered'] * filings_df['Avg_price']

# Filter for 2023 filings on Fridays
friday_filings_2023 = filings_df[(filings_df['Filing Date'].dt.year == 2023) & (filings_df['Filing Date'].dt.dayofweek == 4)]

# Calculate total sum in millions (round to nearest integer)
total_sum_millions = round(friday_filings_2023['Shares_offered_value'].dropna().sum() / 1e6, 0)

print(f"Total sum of 2023 filings on Fridays ($m): {total_sum_millions}")

# Additional info - Check for expected number of records
print(f"Total number of records: {len(filings_df)}")
print(f"Number of records with non-null 'Shares_offered_value' on Fridays in 2023: {len(friday_filings_2023)}")


Total sum of 2023 filings on Fridays ($m): 276.0
Total number of records: 329
Number of records with non-null 'Shares_offered_value' on Fridays in 2023: 32


Answer of Question 1 : 276

# Solution of Question 2

In [2]:
import pandas as pd
import requests
import yfinance as yf
from datetime import datetime, timedelta

# Retrieve IPO data for 2023 and 2024
ipo_urls = ["https://stockanalysis.com/ipos/2023/", "https://stockanalysis.com/ipos/2024/"]
ipo_dfs = []

for url in ipo_urls:
    response = requests.get(url)
    ipo_dfs.append(pd.read_html(response.text)[0])

ipo_df = pd.concat(ipo_dfs).reset_index(drop=True)

# Convert 'IPO Date' to datetime without warning
ipo_df['IPO Date'] = pd.to_datetime(ipo_df['IPO Date']).copy()

# Filter IPOs before March 1, 2024
ipo_df = ipo_df[ipo_df['IPO Date'] < '2024-03-01']

# Get ticker symbols
ticker_symbols = ipo_df['Symbol'].tolist()

# Remove ticker symbol of RYZB
ticker_symbols.remove('RYZB')

# Adjust ticker symbols if needed
ticker_adjustments = {'IBAC': 'IBACU', 'PTHR': 'PTHRF'} # Add adjustments as needed
for i, symbol in enumerate(ticker_symbols):
    if symbol in ticker_adjustments:
        ticker_symbols[i] = ticker_adjustments[symbol]


# Get OHLCV data for each ticker
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')  # 1 year of data
end_date = datetime.now().strftime('%Y-%m-%d')

prices = yf.download(ticker_symbols, start=start_date, end=end_date)

# Calculate growth for each holding period
holding_periods = range(1, 31)
growth_data = {}

for ticker in ticker_symbols:
    try:
        for days in holding_periods:
            initial_price = prices['Adj Close'][ticker].iloc[0]
            final_price = prices['Adj Close'][ticker].iloc[days]
            growth = (final_price - initial_price) / initial_price
            if (ticker, days) not in growth_data:
                growth_data[(ticker, days)] = []
            growth_data[(ticker, days)].append(growth)
    except KeyError:
        print(f"No data found for ticker: {ticker}")

# Create DataFrame from growth data
rows = []
for key, values in growth_data.items():
    for value in values:
        rows.append((key[0], key[1], value))
growth_df = pd.DataFrame(rows, columns=['Ticker', 'Days', 'Growth'])

# Calculate 75% quantile growth for each X
quantile_75 = growth_df.groupby('Days')['Growth'].quantile(0.75)

# Find X where 75% quantile growth is highest
optimal_X = quantile_75.idxmax()
highest_growth = quantile_75.max()
# optimal_X, highest_growth

print ("\nThe optimal number of days X (between 1 and 30), where 75% quantile growth is the highest : ", optimal_X)

[*********************100%%**********************]  184 of 184 completed



The optimal number of days X (between 1 and 30), where 75% quantile growth is the highest :  29


Answer of Question 2 : 29

# Solution of Question 3

In [1]:
import yfinance as yf
import pandas as pd

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM']
EU_STOCKS = ['NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS', 'HINDUNILVR.NS', 'LT.NS']

largest_stock = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA', 'WMT', 'XOM', 'UNH', 'MA', 'PG', 'JNJ', 'MRK', 'HD', 'COST', 'ORCL']
NEW_EU = ['PRX.AS', 'CDI.PA', 'AIR.PA', 'SU.PA', 'ETN', 'SNY', 'BUD', 'DTE.DE', 'ALV.DE', 'MDT', 'AI.PA', 'EL.PA']
NEW_INDIA = ['BAJFINANCE.NS', 'MARUTI.NS', 'HCLTECH.NS', 'TATAMOTORS.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'ADANIENT.NS', 'ADANIENT.NS', 'NTPC.NS', 'KOTAKBANK.NS', 'TITAN.NS']

large_stock = NEW_EU + NEW_US + NEW_INDIA

# Now all_stocks includes both LARGEST & LARGE STOCKS
all_stocks = largest_stock + large_stock

# Download OHLCV data for all stocks for 10 years
start_date = '2014-01-01'
end_date = '2023-12-31'
data_largest = yf.download(largest_stock, start=start_date, end=end_date)
data_large = yf.download(large_stock, start=start_date, end=end_date)
data = yf.download(all_stocks, start=start_date, end=end_date)

## Largest data
# Calculate daily growth for the last 7 days for each stock
def calculate_growth_7d_largest(data_largest):
    return data_largest['Adj Close'].pct_change(7)  # Calculate pct change for last 7 days

# Calculate 7-day growth for all stocks using vectorized operations
growth_7d_largest = calculate_growth_7d_largest(data_largest)

# Combine the 'growth_7d' data with the DataFrame
growth_7d_largest.columns = pd.MultiIndex.from_product([['growth_7d_largest'], growth_7d_largest.columns])
data_largest = pd.concat([data_largest, growth_7d_largest], axis=1)

## Large data
# Calculate daily growth for the last 7 days for each stock
def calculate_growth_7d_large(data_large):
    return data_large['Adj Close'].pct_change(7)  # Calculate pct change for last 7 days

# Calculate 7-day growth for all stocks using vectorized operations
growth_7d_large = calculate_growth_7d_large(data_large)

# Combine the 'growth_7d' data with the DataFrame
growth_7d_large.columns = pd.MultiIndex.from_product([['growth_7d_large'], growth_7d_large.columns])
data_large = pd.concat([data_large, growth_7d_large], axis=1)

## All data
# Calculate daily growth for the last 7 days for each stock
def calculate_growth_7d(data):
    return data['Adj Close'].pct_change(7)  # Calculate pct change for last 7 days

# Calculate 7-day growth for all stocks using vectorized operations
growth_7d = calculate_growth_7d(data)

# Combine the 'growth_7d' data with the DataFrame
growth_7d.columns = pd.MultiIndex.from_product([['growth_7d'], growth_7d.columns])
data = pd.concat([data, growth_7d], axis=1)


# Calculate average daily growth for Largest and Large stocks
try:
    daily_growth_largest = data_largest['growth_7d_largest'].mean(axis=1)
    daily_growth_large = data_large['growth_7d_large'].mean(axis=1)

    # Calculate the number of days when Large outperforms Largest
    outperformance_days = (daily_growth_large > daily_growth_largest).sum()

    # Get the total number of trading days
    total_days = len(daily_growth_large)

    # Calculate the percentage of days Large outperforms Largest (rounded to nearest integer)
    percentage_outperformance = int((outperformance_days / total_days) * 100)
    print(f"\nLarge stocks outperformed Largest stocks on approximately {percentage_outperformance}% of days.")

except KeyError:
    print("Error: 'growth_7d' column not found.")


[*********************100%%**********************]  33 of 33 completed
[*********************100%%**********************]  33 of 33 completed
[*********************100%%**********************]  65 of 65 completed



Large stocks outperformed Largest stocks on approximately 46% of days.


Answer of Question 3: 46

# Solution of Question 4
**See continue of Module2_Colab_Working_with_the_data.ipynb (after Code snippet 9)**

Answer of Question 4 : 2

# Solution of Question 5

Refining the Approach to IPO Investing
Since simply investing in all IPOs isn't a winning strategy, here's how we can refine our approach:

1. Focus on Specific Verticals:

Data: Look for historical IPO data that includes the company's industry sector (verticals). Sources like telecommunications, financial websites or market research firms might offer this data.
Analysis: Analyze which industry sectors have a higher success rate (positive returns for investors post-IPO). This could involve calculating average returns by sector over a specific period.
2. Smart Comparison with Existing Stocks:

Data: We'll need financial data for both IPO companies and their established competitors. This data could include metrics like Price-to-Earnings (P/E) ratio, revenue growth, and market capitalization. Public financial databases or financial news websites can be sources for this data.
Analysis: Compare the valuation (e.g., P/E ratio) of the IPO company to its established competitors. Look for companies with a strong track record, but a lower valuation than their peers. This might indicate higher future growth potential for the IPO.
3. Focus on Company Fundamentals (beyond size):

Data: Look for IPO prospectuses, financial statements, and news articles. These can provide insights into the company's:
Revenue and Profit Growth: Look for companies with consistent and sustainable growth in revenue and profits.
Market Opportunity: Analyze the total addressable market (TAM) for the company's product or service. A large and growing TAM indicates significant potential for future growth.
Competitive Advantage: Does the company have a strong moat, a competitive edge that protects it from rivals? This could be brand recognition, intellectual property, or a unique technology.
Management Team: Evaluate the experience and track record of the management team. A strong leadership team is crucial for navigating future challenges and growth.
Bypassing "Total Number of People":

While the total number of employees might be interesting, it's not a strong indicator of a successful IPO. Instead, focusing on the quality and experience of the team is more relevant.